# Quandl Bulk Fetch and Data Conversion Script
### David Lowe
### August 31, 2021

Main URL: https://www.quandl.com/api/v3/datatables/SHARADAR/SEP?qopts.export=true&api_key=[Key]

In [1]:
!pip install python-dotenv PyMySQL

     |████████████████████████████████| 43 kB 623 kB/s 


In [2]:
import os
import sys
import time
import requests
import zipfile
import pandas as pd
from datetime import date, datetime
from dotenv import load_dotenv

In [3]:
startTimeScript = datetime.now()

In [4]:
# Set up the parent directory location for loading the dotenv files
from google.colab import drive
drive.mount('/content/gdrive')
gdrivePrefix = '/content/gdrive/My Drive/Quandl_Bulk_Fetch/'
env_path = '/content/gdrive/My Drive/Colab Notebooks/'
dotenv_path = env_path + "python_script.env"
load_dotenv(dotenv_path=dotenv_path)

Mounted at /content/gdrive


True

## Task 1) Set up and Retrieve Files from Quandl

In [5]:
api_key = os.environ.get('QUANDL_API')
tables = ['SEP','SFP','ACTIONS','INDICATORS','TICKERS']
as_of_date = datetime.now().date()
# as_of_date = date(2021, 8, 31)

In [6]:
def bulk_fetch(table_name):
    dest_file = gdrivePrefix + 'SHARADAR_' + table_name + '_' + as_of_date.strftime('%Y%m%d') + '.zip'
    url = 'https://www.quandl.com/api/v3/datatables/SHARADAR/%s.json?qopts.export=true&api_key=%s' % (table, api_key)
    resp = requests.get(url)

    valid = ['fresh','regenerating']
    invalid = ['generating']
    status = ''

    while status not in valid:
        resp_dict = resp.json()
        print(resp_dict)
        last_refreshed_time = resp_dict['datatable_bulk_download']['datatable']['last_refreshed_time']
        status = resp_dict['datatable_bulk_download']['file']['status']
        file_link = resp_dict['datatable_bulk_download']['file']['link']
        print('Table', table_name, 'is', status, 'as of', last_refreshed_time)
        if status not in valid:
          time.sleep(60)

    print('fetching from %s' % file_link)
    resp = requests.get(file_link, stream=True)
    with open(dest_file, 'wb') as fd:
        for chunk in resp.iter_content(chunk_size=128):
            fd.write(chunk)
    print(dest_file, 'has been fetched')

In [7]:
for table in tables:
    bulk_fetch(table)

{'datatable_bulk_download': {'file': {'link': 'https://quandl-production-datahub.s3.amazonaws.com/export/SHARADAR/SEP/SHARADAR_SEP_2_0bd2000858d1d8d1f48d4cdea5f8c9e2.zip?X-Amz-Expires=1800&X-Amz-Date=20210831T215923Z&X-Amz-Security-Token=IQoJb3JpZ2luX2VjEKb//////////wEaCXVzLWVhc3QtMSJIMEYCIQCVPd5K7GkiUuxHql4K8AOEYec7Y%2BXIKU8ws9dBnUmWZgIhAJjwFY%2Bj1dyaN0ApWMx2zbL9nLJmXbG049rZVt3lwZcaKoMECN7//////////wEQAhoMMzA2MzIwNTAwMjA0IgxueV2iUzmij6irInYq1wPuIf/XwsM0Qzp2hKYo0rgGHgGIXeuyX%2BdKoArrO2kjl4pofu4jjThSXGukkFXJ2YOP5fPTMSMutaZXfEwoqoaeY2zJdo/gxls3XhkU7iR7RzYaGM4p%2BuMiW/94URwn2fq/Y8VoUilcNLThGsVKNxozqO14HHZ5nEZRxN3tgPozNHZJqyrPKFkAZac%2B9/uTy5HOCaTNhy2io4BM61fvUfFvKuQm17tEUHW7NU1h0kTD/EIeSDOmBtxPEs%2Brp7oh52crEg48i0ZEVAHlY69FvdSaIhOl7FAhaC%2Bx6DJR0g2OwmUxybPmUx7jdIIaqnyLrwudeey/ky8NhSA0InlY1OPtOJwpgHKxyxKZk0Ia6Rot3JIboWCUxKnJvKbXc/y8uYBsqyPvUa4jtfGrBUgEAskaSeaUZx7VW8TrQwR2AhGnQ5bdq4bTXN50gBVXK4s%2BEF9/Q%2BtoTRmYUC%2BakTVbl80IARU3NiBdiiWiVmdlykzbNKK8IvPCUkbnWz5LVpfQU48BqEZHBQXmyM01p1/3vsYtGL

## Task 2) Transform Data from Quandl for Import into Fund Manager

In [8]:
SEP_ZIP_FILE = gdrivePrefix + 'SHARADAR_' + 'SEP' + '_' + as_of_date.strftime('%Y%m%d') + '.zip'
SFP_ZIP_FILE = gdrivePrefix + 'SHARADAR_' + 'SFP' + '_' + as_of_date.strftime('%Y%m%d') + '.zip'
if os.path.exists(SEP_ZIP_FILE) and os.path.exists(SFP_ZIP_FILE):
    print('Retrieved SEP dataset:', SEP_ZIP_FILE)
    print('Retrieved SFP dataset:', SFP_ZIP_FILE)
else:
    sys.exit('Either SEP or SFP data files was not available. Script processing cannot continue!!!')

Retrieved SEP dataset: /content/gdrive/My Drive/Quandl_Bulk_Fetch/SHARADAR_SEP_20210831.zip
Retrieved SFP dataset: /content/gdrive/My Drive/Quandl_Bulk_Fetch/SHARADAR_SFP_20210831.zip


In [9]:
!rm -rf '/content/gdrive/My Drive/Quandl_Bulk_Fetch/FM_interface/'
!mkdir '/content/gdrive/My Drive/Quandl_Bulk_Fetch/FM_interface/'

In [10]:
FM_INTERFACE_DIR = gdrivePrefix + 'FM_interface/'
zip_sep = zipfile.ZipFile(SEP_ZIP_FILE, 'r')
zip_sep.extractall(FM_INTERFACE_DIR)
zip_sep.close()
zip_sfp = zipfile.ZipFile(SFP_ZIP_FILE, 'r')
zip_sfp.extractall(FM_INTERFACE_DIR)
zip_sfp.close()

In [11]:
if len(os.listdir(FM_INTERFACE_DIR)) == 2:
    raw_file_1 = FM_INTERFACE_DIR + os.listdir(FM_INTERFACE_DIR)[0]
    print('Found', raw_file_1, 'for processing...')
    raw_file_2 = FM_INTERFACE_DIR + os.listdir(FM_INTERFACE_DIR)[1]
    print('Found', raw_file_2, 'for processing...')
    df_dataset_import = pd.concat(map(pd.read_csv, [raw_file_1, raw_file_2]), ignore_index=True)
    df_dataset_import['date'] = pd.to_datetime(df_dataset_import['date'])
    df_dataset_import.drop(columns=['close','closeunadj','lastupdated'], inplace=True)
    df_dataset_import.sort_values(by=['date', 'ticker'], ascending=True, inplace=True, ignore_index=True)
    print()
    print(df_dataset_import.head())
    print()
    print(df_dataset_import.tail())
    print()
    df_dataset_import.info()
else:
    sys.exit('Not all data files are available. Script processing cannot continue!!!')

Found /content/gdrive/My Drive/Quandl_Bulk_Fetch/FM_interface/SHARADAR_SEP_2_0bd2000858d1d8d1f48d4cdea5f8c9e2.csv for processing...
Found /content/gdrive/My Drive/Quandl_Bulk_Fetch/FM_interface/SHARADAR_SFP_2_57b147f453795a51de647236d26a618c.csv for processing...

  ticker       date   open    high     low     volume  closeadj
0      A 2011-01-03  41.56  42.140  41.411  3572300.0    27.710
1   AACC 2011-01-03   5.99   6.250   5.990    20400.0     6.150
2   AACG 2011-01-03   3.67   3.760   3.670     2400.0     1.138
3  AACPF 2011-01-03   9.70   9.700   9.630     1160.0     9.630
4   AADR 2011-01-03  29.98  30.063  29.880     1600.0    28.656

         ticker       date      open  ...       low        volume  closeadj
23319935   ^DJI 2021-08-30  35471.80  ...  35374.46  2.453885e+08  35399.84
23319936  ^GSPC 2021-08-30   4513.76  ...   4513.76  1.560943e+09   4528.79
23319937  ^IXIC 2021-08-30  15165.94  ...  15165.12  3.730925e+09  15265.89
23319938   ^RUT 2021-08-30   2278.45  ...   22

In [12]:
df_dataset_import['date'] = df_dataset_import['date'].dt.strftime('%m/%d/%y')
new_order = ['ticker', 'date', 'closeadj', 'open', 'high', 'low', 'volume']
df_dataset_import = df_dataset_import.reindex(columns=new_order)
print(df_dataset_import.head())
print()
print(df_dataset_import.tail())

  ticker      date  closeadj   open    high     low     volume
0      A  01/03/11    27.710  41.56  42.140  41.411  3572300.0
1   AACC  01/03/11     6.150   5.99   6.250   5.990    20400.0
2   AACG  01/03/11     1.138   3.67   3.760   3.670     2400.0
3  AACPF  01/03/11     9.630   9.70   9.700   9.630     1160.0
4   AADR  01/03/11    28.656  29.98  30.063  29.880     1600.0

         ticker      date  closeadj      open      high       low        volume
23319935   ^DJI  08/30/21  35399.84  35471.80  35510.71  35374.46  2.453885e+08
23319936  ^GSPC  08/30/21   4528.79   4513.76   4537.36   4513.76  1.560943e+09
23319937  ^IXIC  08/30/21  15265.89  15165.94  15288.08  15165.12  3.730925e+09
23319938   ^RUT  08/30/21   2265.99   2278.45   2286.36   2264.30  0.000000e+00
23319939   ^VIX  08/30/21     16.19     16.77     16.80     15.98  0.000000e+00


In [13]:
output_file = FM_INTERFACE_DIR + 'SHARADAR_SEP_SFP.csv'
try:
    df_dataset_import.to_csv(output_file, sep=',', header=False, index=False)
    print('Data frame converted successfully to the output file.')
except:
    print('Data frame did not get converted to the output file!')

Data frame converted successfully to the output file.


In [14]:
print ('Total time for the script:',(datetime.now() - startTimeScript))

Total time for the script: 0:07:18.330447
